In [7]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 12991 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_2_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [11]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10, 20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=996764, Wed Apr 15 14:04:01 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.653523284301128e+08 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.630101344692093e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.85504658e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.92976349e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.572888774980397e+08 1.0e+00 2.33e+00  2e+00  2e+00 0:00.4
   29    551 2.502880530436808e+07 1.1e+00 2.00e+00  2e+00  2e+00 0:03.4
   64   1216 2.511210098947465e+07 1.1e+00 1.90e+00  2e+00  2e+00 0:07.5
  100   1900 -1.505466837410569e+07 1.2e+00 1.67e+00  2e+00  2e+00 0:11.7
  152   2888 -8.509293433781523e+07 1.2e+00 1.28e+00  1e+00  1e+00 0:17.8
  200   3800 -1.747373640940606e+08 1.3e+00 8.73e-01  8e-01  9e-01 0:23.5
  267   5073 -2.752847758885700e+08 1.3e+00 4.40e-01  4e-01  4e-01 0:31.5
  300   5700 -2.992913346183776e+08 1.3e+00 3.16e-01  3e-01  3e-01 0:35.5
  385   7315 -3.205199352734936e+08 1.3e+00 1.53e-01  1e-01  2e-01 0:45.5
  400   7600 -3.218921636142058e+08 1.3e+00 1.32e-01  1e-01  1e-01 0:47.2
  500   9500 -3.262427209657421e+08 1.3e+00 5.36e-02  5e-02  5e-02 0:59.0
  600  11400 -3.268737277705240e+08 1.3e+00 2.19e-02  2e-02  2e-02 1:10.8
  699  13281 -3.269920453660606e+08 1.3e+00 8.70e-03  8e-03  9e-03 1:24.9
  700  13300 -3.269912429930844e+08 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Wed Apr 15 14:26:43 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [-4.97841244  3.02657454  2.44845895 -1.58886696  0.67657797  4.42223407
  1.64672906  0.43086725 ...]
std deviations: [2.43734162 2.43717638 2.43737485 2.43786168 2.43797938 2.43759995
 2.43750368 2.43778116 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1045958, Wed Apr 15 14:26:43 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.647390442849424e+08 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.717783348456163e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

   27    513 3.350968357828015e+07 1.1e+00 2.00e+00  2e+00  2e+00 0:03.5
   59   1121 -8.322639274905622e+06 1.1e+00 1.82e+00  2e+00  2e+00 0:07.7
  100   1900 -1.341174376971623e+07 1.2e+00 1.57e+00  2e+00  2e+00 0:12.7
  149   2831 -9.355660414883739e+07 1.2e+00 1.17e+00  1e+00  1e+00 0:18.8
  200   3800 -2.112114339209359e+08 1.2e+00 7.65e-01  7e-01  8e-01 0:24.6
  268   5092 -2.928406731078668e+08 1.3e+00 3.69e-01  4e-01  4e-01 0:32.7
  300   5700 -3.087478226034459e+08 1.3e+00 2.72e-01  3e-01  3e-01 0:36.6
  382   7258 -3.220457168223690e+08 1.3e+00 1.31e-01  1e-01  1e-01 0:46.7
  400   7600 -3.233540061188552e+08 1.3e+00 1.12e-01  1e-01  1e-01 0:48.9
  498   9462 -3.264435061049398e+08 1.3e+00 4.67e-02  4e-02  5e-02 1:01.0
  500   9500 -3.264465099335350e+08 1.3e+00 4.59e-02  4e-02  5e-02 1:01.3
  600  11400 -3.269196165724081e+08 1.3e+00 1.77e-02  2e-02  2e-02 1:13.4
  700  13300 -3.270008768592035e+08 1.3e+00 7.21e-03  7e-03  7e-03 1:25.7
  800  15200 -3.270115063433967e+08 1.3

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.02915659e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=202915658.700625, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Wed Apr 15 20:54:18 2020)
final/bestever f-value = 2.029157e+08 2.029157e+08
incumbent solution: [-4.98672091 -4.9998943   4.99436297  4.84108464  4.31650913 -1.15141339
  3.97550454 -3.19262333 ...]
std deviations: [2.43708916 2.43677318 2.43699235 2.43686669 2.43680967 2.43668738
 2.43671693 2.43657001 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1124616, Wed Apr 15 20:54:19 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.795558132525402e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.344010150673805e+08 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2.66566761e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.85513812e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.615127857139786e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 5.451915740438494e+07 1.1e+00 2.07e+00  2e+00  2e+00 0:03.4
   69   1311 4.155024563217890e+07 1.2e+00 1.89e+00  2e+00  2e+00 0:07.5
  100   1900 4.365219990822622e+07 1.2e+00 1.79e+00  2e+00  2e+00 0:10.8
  156   2964 4.013329454359712e+07 1.3e+00 1.73e+00  2e+00  2e+00 0:16.9
  200   3800 3.130522951097815e+07 1.3e+00 1.63e+00  2e+00  2e+00 0:21.6
  274   5206 2.693892530579405e+07 1.4e+00 1.28e+00  1e+00  1e+00 0:29.6
  300   5700 2.342485360831334e+07 1.4e+00 1.14e+00  1e+00  1e+00 0:32.4
  394   7486 1.585042035199527e+07 1.4e+00 8.07e-01  8e-01  8e-01 0:42.5
  400   7600 1.249710275299673e+07 1.4e+00 7.85e-01  7e-01  8e-01 0:43.1
  500   9500 8.835737612424791e+06 1.5e+00 5.26e-01  5e-01  5e-01 0:53.8
  600  11400 6.474960600223407e+06 1.5e+00 3.96e-01  4e-01  4e-01 1:04.5
  700  13300 5.232341422706842e+06 1.5e+00 3.15e-01  3e-01  3e-01 1:15.4
  800  15200 3.549614110321805e+06 1.5e+00 2.10e-01

In [13]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:200])
    SVM_Fun [i] = F2(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F2(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([50672204.57925562]), count=array([1])) 0.3427357414838865
SVM
ModeResult(mode=array([1.44507209e+08]), count=array([1])) 49250060.29749079
ELN
ModeResult(mode=array([3294399.92636666]), count=array([1])) 312564.83185352007
